In [1]:
import os, cv2, numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import joblib

''' Manually label raindrop and clean pictures. Save them in the folder structure shown below'''
# --- Path to folders with clean and raindrop imaged ---
CLEAN_DIR = "data/raindrop_clean_split/clean"
RAIN_DIR  = "data/raindrop_clean_split/raindrop"

base = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

def embedding_from_path(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # centre-crop 1944×1944 → resize to 224×224
    h, w = img.shape[:2]
    c = min(h, w)
    y0 = (h - c)//2; x0 = (w - c)//2
    crop = img[y0:y0+c, x0:x0+c]
    # resize to MobileNet input
    img = cv2.resize(crop, (224, 224)).astype("float32")
    x = preprocess_input(img[None, ...])
    feat = base.predict(x, verbose=0)
    return feat.flatten() # returns 1280-dimensional feature vector. 

# --- Building dataset of feature vectors ---
X, y = [], []
for folder, label in [(CLEAN_DIR, 0), (RAIN_DIR, 1)]:
    for f in os.listdir(folder):
        if f.lower().endswith((".jpg",".jpeg",".png")):
            X.append(embedding_from_path(os.path.join(folder, f)))
            y.append(label)
X, y = np.array(X), np.array(y)
print("Feature matrix shape:", X.shape)
# Feature matrix shape = (1744, 1280)

# --- Train SVM --
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC(kernel="linear", probability=True)
clf.fit(X_train, y_train)

pred = clf.predict(X_val)
print(classification_report(y_val, pred, target_names=["clean", "raindrop"]))


# --- Save models ---
# Save SVM to a file
joblib.dump(clf, "raindrop_svm.pkl")
print("SVM model saved as raindrop_svm.pkl")

# Save embedder to file. 
base.save("mobilenetv2_embedder.keras")
print("Feature extractor saved as mobilenetv2_embedder.keras")

C:\Users\tisha\AppData\Local\Temp\ipykernel_27308\3972278667.py:14: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


Feature matrix shape: (1744, 1280)
              precision    recall  f1-score   support

       clean       1.00      0.99      0.99       179
    raindrop       0.99      1.00      0.99       170

    accuracy                           0.99       349
   macro avg       0.99      0.99      0.99       349
weighted avg       0.99      0.99      0.99       349

